Nambu superconducting single-orbital Hubbard model
============================

In this notebook, we generalize the Bethe lattice DMFT on the one-band Hubbard model to the Nambu basis.
This allows to compute quantities in the superconducting phase, like the anomalous Green function called the Gorkov function $F(i\omega_n) = \int_0^\beta d\tau e^{i\omega_n\tau} \langle c_{\uparrow}(\tau) c_{\downarrow} \rangle$.
To bench mark these calculations, we study the $U < 0$ case where s-wave superconductivity is promoted.

In the Nambu basis, we define the Nambu spinors as
$$
\Psi = \left( \begin{matrix} c_{\uparrow} \\ c^{\dagger}_{\downarrow} \end{matrix} \right)
\quad \text{and} \quad
\Psi^{\dagger} = \left( \begin{matrix} c^{\dagger}_{\uparrow} & c_{\downarrow} \end{matrix} \right).
$$
In this basis, the Nambu Green function in imaginary time reads as
$$
\mathcal{G}(\tau) = - \langle T_{\tau} \Psi(\tau) \Psi^{\dagger} \rangle = \left(
    \begin{matrix} G(\tau) & F(\tau) \\ F^*(\tau) & - G(-\tau) \end{matrix} \right).
$$
In Matsubara frequencies, we have
$$
\mathcal{G}(i\omega_n) =
    \left( \begin{matrix} G(i\omega_n) & F(i\omega_n) \\ F^*(-i\omega_n) & - G(-i\omega_n) \end{matrix} \right) =
    \left( \begin{matrix} G(i\omega_n) & F(i\omega_n) \\ F^*(-i\omega_n) & - G^*(i\omega_n) \end{matrix} \right).
$$
The Hubbard interaction  mediating pairing is
$$
H_{\text{int}} = U c^{\dagger}_{\uparrow} c_{\uparrow} c^{\dagger}_{\downarrow} c_{\downarrow}
    = U \Psi_0^{\dagger} \Psi_0 \Psi_1 \Psi_1^{\dagger}.
$$

In the Bethe lattice, the non-interacting Green's function has a semi-circular density of state and the self-consistency equation relating $\mathcal{G}_0$ to the interacting matrix Green's function is
$$
\mathcal{G}_0^{-1}(i\omega_n) = i\omega_n + \mu \sigma_3 - t^2 \sigma_3 \mathcal{G}(i\omega_n) \sigma_3
$$
where $\sigma_3$ is the third Pauli matrix in particle-hole space.


In [ ]:
from triqs.gf import *
from triqs.operators import *
from h5 import *
from triqs_cthyb import Solver
from numpy import *
from triqs.plot.mpl_interface import *
import numpy as np

from itertools import product as itp

U = -.2
t = 1
niter = 50
mu = 0
Delta0 = 0.1

# h_int = U * n('nambu', 0) * n('nambu', 1)
h_int = U * n('nambu', 0) * c('nambu', 1)*c_dag('nambu', 1)

plt.figure(figsize=(12,8))

for beta in [5, 20, 50, 100, 200]:
# for beta in [200]:
    S = Solver(beta = beta, gf_struct = [('nambu', [0, 1])])

    p = {'n_warmup_cycles': 5000,
         'n_cycles': 1000000}

    s3 = np.matrix([[1, 0], [0, -1]])

    Sigma = S.G0_iw['nambu'].copy()
    Sigma.zero()
    Sigma[0, 1] << Delta0
    Sigma[1, 0] << Delta0

    S.G0_iw['nambu'][0, 0] << SemiCircular(2*t)
    S.G0_iw['nambu'][1, 1] = S.G0_iw['nambu'][0, 0].copy()
    S.G0_iw['nambu'][1, 1] << -1 * S.G0_iw['nambu'][1, 1].conjugate()

    S.G_iw['nambu'] << inverse(inverse(S.G0_iw['nambu']) - Sigma)
    
    max_Sigma01 = []

    for i in range(niter):
        print("########## Beta = %.2f ##### Iteration = %i" % (beta, i))
        # S.G0_iw['nambu'] << iOmega_n - mu*s3 - t**2 * s3.dot(S.G_iw['nambu']).dot(s3)
        S.G0_iw['nambu'][0, 0] << iOmega_n + mu - t**2 * S.G_iw['nambu'][0, 0]
        S.G0_iw['nambu'][0, 1] << + t**2 * S.G_iw['nambu'][0, 1]
        S.G0_iw['nambu'][1, 0] << + t**2 * S.G_iw['nambu'][1, 0]
        S.G0_iw['nambu'][1, 1] << iOmega_n - mu - t**2 * S.G_iw['nambu'][1, 1]
        S.G0_iw['nambu'].invert()
        S.solve(h_int = h_int, **p)

        Sigma = GfImFreq(beta = beta, indices = [0, 1])
        Sigma << inverse(S.G0_iw['nambu']) - inverse(S.G_iw['nambu'])
        
        with HDFArchive("results_bethe_SC_beta%i_U%.2f_Delta%.2f.h5" % (beta, U, Delta0), 'a') as A:
            A['G_iw-iter%i' % i] = S.G_iw
            A['Sigma-iter%i' % i] = Sigma
        
        max_Sigma01.append(abs(Sigma[0, 1](0)))
    
    plt.plot(np.arange(niter)[1:], max_Sigma01[1:], label=r'$\beta=%d$' % beta)
    # oplot(S.G_iw['nambu'][1, 1].imag, label=r'$\beta=%i$' % beta)
    
plt.legend()
# plt.xlim(-50, 50)
plt.show()

In [ ]:
oplot(Sigma[1, 0])
# oplot(S.G_iw["nambu"][0,0])

In [ ]:
g_real = GfReFreq(indices=[0, 1], window=(-10, 10))
g_real.set_from_pade(S.G_iw["nambu"])
oplot(-g_real.imag/np.pi)